# Install and import required packages

In [1]:
!pip install -q -U grain clu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.5/494.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.9 MB/s eta 0:00:00


In [3]:
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

'''
    Uncomment the line bellow to simulate 8 devices when running on CPU.
'''
flags = os.environ.get("XLA_FLAGS", "")
flags += " --xla_force_host_platform_device_count=8"  # Simulate 8 devices
os.environ["XLA_FLAGS"] = flags

import jax
from jax import numpy as jnp
from flax import linen as nn
from flax.training import train_state
from flax.training.common_utils import shard, shard_prng_key, get_metrics
import optax
import numpy as np
import grain.python as grain
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ml_collections
from tqdm.auto import tqdm
from clu import metric_writers
import orbax.checkpoint as ocp
from tokenizers import ByteLevelBPETokenizer

import pathlib
import requests
import zipfile

prng_main_key = jax.random.key(18)

print(f"Jax Device count: {jax.device_count()}")

Jax Device count: 8


In [ ]:
# Import custom files

# Download the code to the current working directory
REPO_NAME = 'transformer-from-scratch'
if not os.path.isdir(REPO_NAME):
    !git clone https://github.com/MiguelSteph/transformer-from-scratch.git

%run 'transformer-from-scratch/configs/configs.py'
%run 'transformer-from-scratch/configs/dataset_utils.py'

config = get_configs()

# Download the dataset
We will use the french to english dataset.
It can be downloaded at https://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip

In [21]:
download_data(config)